Follows $\texttt{https://github.com/ericmetodiev/OmniFold/blob/master/OmniFold\%20Demo.ipynb}$ but uses pip-installable version of OmniFold

In [1]:
import numpy as np
from matplotlib import pyplot as plt

import energyflow as ef # for data
import omnifold # for omnifold algorithm

# Local imports for plotting (source: https://github.com/ericmetodiev/OmniFold/blob/master/modplot.py)
import modplot
from plot_utils import *

plt.rcParams['figure.figsize'] = (4,4)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.family'] = 'serif'

# Global variables
OBS_MULTIFOLD = ['Mass', 'Mult', 'Width', 'Tau21', 'zg', 'SDMass']
N_DATA = int(1e6)


2025-03-23 15:58:01.658697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742770681.676315  477008 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742770681.681540  477008 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742770681.697682  477008 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742770681.697699  477008 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742770681.697701  477008 computation_placer.cc:177] computation placer alr

Horovod not found, will continue with single only GPUs.


# Load data

Choose Pythia as the Monte Carlo data

In [ ]:
data_mc = ef.zjets_delphes.load('Pythia26', num_data=n_data, exclude_keys=['particles'])
data_mc_gen = {k.replace("gen_", ""): data_mc[k] for k in data_mc.keys() if 'gen_' in k}
data_mc_sim= {k.replace("sim_", ""): data_mc[k] for k in data_mc.keys() if 'sim_' in k}

data_mc_gen['Mass'] = data_mc_gen['jets'][:,3]
data_mc_gen.pop('jets')
data_mc_gen['Mult'] = data_mc_gen.pop('mults')
data_mc_gen['Width'] = data_mc_gen.pop('widths')
data_mc_gen['Tau21'] = data_mc_gen.pop('tau2s')
data_mc_gen['zg'] = data_mc_gen.pop('zgs')
data_mc_gen['SDMass'] = data_mc_gen.pop('sdms')
data_mc_gen = np.concatenate([data_mc_gen[k].reshape() for k in obs_multifold], axis=1)
print(f"data_mc_gen.shape: {data_mc_gen.shape}")

data_mc_sim['Mass'] = data_mc_sim['jets'][:,3]
data_mc_sim.pop('jets')
data_mc_sim['Mult'] = data_mc_sim.pop('mults')
data_mc_sim['Width'] = data_mc_sim.pop('widths')
data_mc_sim['Tau21'] = data_mc_sim.pop('tau2s')
data_mc_sim['zg'] = data_mc_sim.pop('zgs')
data_mc_sim['SDMass'] = data_mc_sim.pop('sdms')

# Create DataLoader objects

AxisError: axis 1 is out of bounds for array of dimension 1

In [34]:
data_mc_gen['Mass'].shape

(1000000,)

In [24]:
data_mc_gen['jets'].shape

(1000000, 4)

In [25]:
data_mc_gen.keys()

dict_keys(['tau2s', 'widths', 'Zs', 'ang2s', 'zgs', 'jets', 'mults', 'sdms', 'lhas'])

And Hedwig as the true/nature data

In [ ]:
datasets = {'Pythia26': ef.zjets_delphes.load('Pythia26', num_data=1000000, exclude_keys=['particles']),
            'Herwig': ef.zjets_delphes.load('Herwig', num_data=1000000, exclude_keys=['particles'])}

# choose what is MC and Data in this context
data_mc, data_nature = datasets['Pythia26'], datasets['Herwig']

# how many iterations of the unfolding process
itnum = 3



In [13]:
data_mc.keys()

dict_keys(['gen_tau2s', 'gen_widths', 'gen_Zs', 'gen_ang2s', 'gen_zgs', 'gen_jets', 'gen_mults', 'gen_sdms', 'gen_lhas', 'sim_tau2s', 'sim_widths', 'sim_ang2s', 'sim_zgs', 'sim_jets', 'sim_mults', 'sim_sdms', 'sim_lhas'])

# Construct DataLoader objects

Let's assume, for simplicity, that everything passes both the reco and gen cuts.

In [6]:
data_mc_gen = {k.replace("gen_", ""): data_mc[k] for k in data_mc.keys() if 'gen_' in k}
data_mc_sim= {k.replace("sim_", ""): data_nature[k] for k in data_nature.keys() if 'sim_' in k}

data_nature_gen = {k.replace("gen_", ""): data_nature[k] for k in data_nature.keys() if 'gen_' in k}
data_nature_sim = {k.replace("sim_", ""): data_nature[k] for k in data_nature.keys() if 'sim_' in k}

Now the dataloader objects: truth, Monte-Carlo

# MultiFold object

In [8]:
name = "MultiFold"
model_reco = omnifold.net.MLP(nvars = len(obs_multifold))
model_gen = omnifold.net.MLP(nvars = len(obs_multifold))
data = {k: nature[k] for k in obs_multifold}
mc = {k: synthetic[k] for k in obs_multifold}

model = omnifold.MultiFold(name=name, model_reco=model_reco, model_gen=model_gen, data=data, mc=mc)


KeyError: 'Mass'